In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav

(rate,sig) = wav.read("english.wav")
mfcc_feat = mfcc(sig,rate)

print(mfcc_feat[1:2,:])

[[ 13.93312459 -23.69237924  -1.57510247   1.31569228   8.04625046
   13.96440739  19.37418855  13.79980361  11.86047765  -3.02382684
   -0.40233984  -4.4249153   -6.63196157]]


In [25]:
from math import exp
from random import seed
from random import random



# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network




# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation


# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))





# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs




# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

# Test training backprop algorithm
seed(1)
dataset = [
    [4.37275458,-4.58897544,-7.71213097,-8.38301615,-29.89204263,1.28124156,-5.12565644,14.94425204,-3.88802572,-9.31919016,-11.44426269,-0.68076128,2.23264512,0],
    [6.63617188e-01,-2.30192371e+01,-2.90644828e+01,-5.28981197e+00,-5.22499269e+00,-1.53603590e+01,3.03658800e+00,-8.41977906e+00,-5.22680640e+00,6.26783773e+00,-1.44572013e+01,1.19444132e+01,-1.31164009e+01,0],
    [2.60972421,-22.23815865,-25.95823821,-7.79926968,-13.54058039,-8.38011547,-10.46343918,-6.04023341,-12.49967038,-2.42185989,-13.36330556,-4.06015751,-6.22552088,0],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,0],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,0],
    [3.69234135e-01,-1.27369724e+01,-1.04838833e+01,-1.08952808e+01,-1.49528915e+01,-5.41837664e+00,2.17393419e+00,2.84333813e+00,-9.73215469e+00,-6.34839707e+00,6.44872034e+00,3.17496373e+00,-1.09868069e+01,0],
    [3.99737784,-12.94697708,-19.72691111,-14.58548374,-15.49956657,13.26430269,-9.63233446,2.95232263,-2.39254547,1.08696274,-0.98483044,-6.81788491,1.27650826,0],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,0],
    [3.73431552,-11.43522862,-15.69738108,-7.23093618,-9.20143783,15.77861198,-8.21196815,-3.45524077,-3.85866026,2.22537973,-6.74939318,-3.4748446,-1.31289862,0],
    [6.63617188e-01,-2.30192371e+01,-2.90644828e+01,-5.28981197e+00,-5.22499269e+00,-1.53603590e+01,3.03658800e+00,-8.41977906e+00,-5.22680640e+00,6.26783773e+00,-1.44572013e+01,1.19444132e+01,-1.31164009e+01,0],
    [13.27591317,30.943853,-5.65019689,-45.8268965,-12.16801497,-9.24776321,2.69744928,-26.26107665,14.11174374,-1.47232874,12.7455087,-28.26788586,-2.2764699,1],
    [7.97980362,-14.93251634,-10.99151692,-8.78721484,5.21320101,-15.26261328,11.23777555,-13.87077327,-3.65886202,-1.89670985,-17.77552902,7.61855015,-14.27392173,1],
    [5.46006947,-15.91929383,-7.26691309,-3.01975139,5.94029413,-1.16676046,-6.8591623,1.22502565,3.56804853,5.0226074,10.26914032,12.97331335,15.19672871,1],
    [6.95538173,3.76749167,0.12484054,-4.89019938,21.64086463,-1.02768468,-10.3509281,-7.9197257,9.17413682,-3.47999443,3.20271527,-17.08076568,-17.07314933,1],
    [12.60767466,14.47032609,-12.69994493,-25.12286409,60.17865954,10.44012883,-8.22777681,-47.09400057,30.39564841,-11.59831444,-10.95849235,-31.85902194,7.24876843,1],
    [10.81613729,25.51449113,-4.06048601,-22.95256932,20.46820577,2.81170631,-16.42161144,-38.13638718,5.26492951,-5.42227484,27.33926829,12.73300632,13.48835487,1],
    [3.75537396,-17.6742283,-17.09203835,-1.16349105,-15.03584179,-2.59776287,-13.27282739,-6.47717668,-12.48057374,4.02514779,-9.62068121,-5.14022815,-13.69378263,1],
    [4.26358282,-10.08142772,-18.70702963,-13.27196589,-8.59244527,11.96732636,-12.18367498,-5.05763032,5.44878225,5.57191464,-1.17010204,1.72100768,2.49546287,1],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,1],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,1],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,2],
    [6.63617188e-01,-2.30192371e+01,-2.90644828e+01,-5.28981197e+00,-5.22499269e+00,-1.53603590e+01,3.03658800e+00,-8.41977906e+00,-5.22680640e+00,6.26783773e+00,-1.44572013e+01,1.19444132e+01,-1.31164009e+01,2],
    [0.66361719,-23.0192371,-29.06448283,-5.28981197,-5.22499269,-15.36035895,3.036588,-8.41977906,-5.2268064,6.26783773,-14.45720127,11.94441319,-13.11640092,2],
    [15.83132882,15.43334614,-28.32736797,-55.55718533,40.49898355,-20.43688216,-6.22255689,-41.7191769,53.03246348,2.86154177,-6.24027092,-35.19025244,12.16122707,2]
]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 8, n_outputs)
train_network(network, dataset, 0.5, 1, n_outputs)

for layer in network:
    print(layer[0])
    print("    ")
    print(network[0][0])
    
        
# # Test making predictions with the network
# dataset = [
#     [3.69234135e-01,-1.27369724e+01,-1.04838833e+01,-1.08952808e+01,-1.49528915e+01,-5.41837664e+00,2.17393419e+00,2.84333813e+00,-9.73215469e+00,-6.34839707e+00,6.44872034e+00,3.17496373e+00,-1.09868069e+01,0],
#     [6.63617188e-01,-2.30192371e+01,-2.90644828e+01,-5.28981197e+00,-5.22499269e+00,-1.53603590e+01,3.03658800e+00,-8.41977906e+00,-5.22680640e+00,6.26783773e+00,-1.44572013e+01,1.19444132e+01,-1.31164009e+01,0],
#     [0.66361719,-22.1039732,-29.20833398,-5.97788713,-4.82966574,-15.60393428,2.50664894,-7.88384146,-7.30642318,6.62174055,-16.10205247,11.30991508,-12.91888839,1],
#     [4.7007528,-8.14552688,-19.15550144,-13.70426643,-7.48215336,6.33313457,-12.65443029,-3.78606738,3.1367897,0.25679685,-1.54501683,-4.50866517,2.46214346,1],
#     [6.63617188e-01,-2.30192371e+01,-2.90644828e+01,-5.28981197e+00,-5.22499269e+00,-1.53603590e+01,3.03658800e+00,-8.41977906e+00,-5.22680640e+00,6.26783773e+00,-1.44572013e+01,1.19444132e+01,-1.31164009e+01,2]
# ]
# network = [
#         [{'weights': [0.13773482570560408, 0.8378974657482752, 0.7593396650087151, 0.2530460250359365, 0.4989946072370689, 0.4487444350529932, 0.6474383654952118, 0.7893575006419559, 0.09608534264323067, 0.031390068783063184, 0.8420524951698254, 0.44052384834135255, 0.7714938685100511, 0.0027174220177603767]}, {'weights': [0.5028366292907667, 0.6487886437457592, 0.34945260634981784, 1.1401520579804483, 1.0435382788164436, 0.13630452608183702, -0.15624605308486395, 0.6034438394672031, 0.7860308883849411, 0.33149318063622407, 0.41238763243114074, 0.48664605547474005, -0.1724481374245239, 0.23752874707274804]}, {'weights': [0.5517119146212391, 0.5383812792547484, 0.05928338304503042, 0.0012760322519545409, 0.5124646705784179, 0.44498961674651655, -0.024999749702042676, -0.28418569516818787, 1.2939935965537896, 0.7167211510722009, 0.7611683517672757, 0.3024738287415049, 0.9413495565532797, 0.8770837023371869]}, {'weights': [0.12088942952327576, 0.33268824243007794, 0.721485840117162, 0.7111973412210513, 0.9364434043644083, 0.42210558462218456, 0.8300365924215267, 0.6703149155104451, 0.30337086102263494, 0.5875808741168979, 0.8824711821203325, 0.8461854071466484, 0.5052715566160174, 0.5890023697352963]}, {'weights': [0.03379635533638255, 0.24104088466266213, 0.7976809815355196, 0.415854469630841, 0.17162039874149823, 0.5486115901371891, 0.7041473624492838, 0.6770477556426512, 0.37434550110517695, 0.43931846746508024, 0.5065796243443019, 0.7775763604819299, 0.5200202509776045, 0.39318737428126943]}, {'weights': [0.23781080605509686, -0.21318438234337192, 0.4906443850284451, 1.579645536116181, 0.3395997861931019, 0.913285943433376, 0.4967227674900377, 0.8294216308885755, -0.33509539251942644, 0.9378145210319266, 0.8674807635618833, 1.0988495583351432, 0.6754771212961904, 0.21596702094469541]}, {'weights': [0.46660394084556794, 0.03478071977698121, -0.10511717248082281, 0.09581919641666237, -1.708845930833645, 0.843010958485126, 0.9214404201207308, 1.08321713370215, 0.034900415835710984, 0.7685614595838679, 1.0155846431358206, 1.2416637961132975, 0.6050174044259147, 0.5669132365920813]}, {'weights': [0.5613959425402698, 0.42610181028027144, 0.05609257703731692, 0.8699598512881519, 0.5701134478501684, 0.1998577609589153, 0.5046665796970163, 0.4848563357875168, 0.35685300769494877, 0.3460627840902753, 0.5384755508444701, 0.6234038187846188, 0.612401445040514, 0.45815206515838086]}],
# [{'weights': [-0.8536671299938119, -1.6424791313830571, -9.610493689648342, -0.2983676816019588, 0.24526883709850042, 0.05335084621870843, 5.172144837040033, -0.06639393363478974, -0.3599457470029537]}, {'weights': [1.0441738025110108, 4.165410873161359, 2.2573187426331374, 0.21919641586438196, 0.061294311288905674, 0.7671995810633003, -0.6373765647303952, 0.31200545993373024, -0.9579592509686934]}, {'weights': [-0.5947607453799076, -5.104672132693334, 1.2405582402637545, -0.41272495069437337, -0.2921340297009941, -0.2704414630470757, -4.9055828620918085, -0.48541422629298797, -1.1145696234605007]}]
#     ]
# for row in dataset:
# 	prediction = predict(network, row)
# 	print('Expected=%d, Got=%d' % (row[-1], prediction))

>epoch=0, lrate=0.500, error=21.777
{'weights': [0.13419815207374644, 0.8479180169972278, 0.7639956824733499, 0.25516088069212983, 0.4952543956545578, 0.4495265567454752, 0.6518016247165801, 0.7886860767216208, 0.09375105296621714, 0.02819468294441735, 0.835452714062573, 0.432372396114503, 0.7618177752461954, 0.0020756336156070795], 'output': 1.1663370485436897e-23, 'delta': -7.492132968057199e-25}
    
{'weights': [0.13419815207374644, 0.8479180169972278, 0.7639956824733499, 0.25516088069212983, 0.4952543956545578, 0.4495265567454752, 0.6518016247165801, 0.7886860767216208, 0.09375105296621714, 0.02819468294441735, 0.835452714062573, 0.432372396114503, 0.7618177752461954, 0.0020756336156070795], 'output': 1.1663370485436897e-23, 'delta': -7.492132968057199e-25}
{'weights': [-0.0016688526462894173, 0.007237359416847278, -0.06320490524687583, 0.5548077319409769, 0.8568174057720926, 0.66479895534826, 0.7084007382500472, 0.7867839006107267, -0.12054080088226857], 'output': 0.6474790759189

[1, 2, 3, [4, 5]]


5
